This demo is designed to work with the standard socket workers alice and bob from the other demos. It adds two numbers together

In [1]:
import syft as sy
import torch

In [2]:
from syft.mpc import spdz
from syft.core.frameworks.torch import _GeneralizedPointerTensor as gpt
from syft.core.frameworks.torch import _MPCTensor as mpct

In [3]:
local_worker = sy.SocketWorker(id="local", port=2008, hook=None, is_client_worker=False)
hook = sy.TorchHook(local_worker=local_worker, verbose=False)
me = hook.local_worker
me.hook = hook

Starting Socket Worker...
Ready to receive commands...


In [4]:
x = torch.FloatTensor([[1,2],[3,4]])
y = torch.FloatTensor([[2,3],[4,5]])

In [5]:
x_enc = spdz.encode(x)
y_enc = spdz.encode(y)

In [6]:
x_alice, x_bob = spdz.share(x_enc)
y_alice, y_bob = spdz.share(y_enc)

In [7]:
x_bob


 1.9412e+09  7.0410e+08
 7.6921e+08  6.2381e+08
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

In [8]:
bob = sy.SocketWorker(id="bob", port=2006, hook=hook, is_pointer=True, is_client_worker=False)

alice = sy.SocketWorker(id="alice", port=2007, hook=hook, is_pointer=True, is_client_worker=False)

Attaching Pointer to Socket Worker...
Attaching Pointer to Socket Worker...


In [9]:
compute_nodes = [bob, alice]

me.add_workers([bob, alice])

In [10]:
bob.add_workers([me, alice])
alice.add_workers([me, bob])

In [11]:
x_alice.send(alice)
x_bob.send(bob)
y_alice.send(alice)
y_bob.send(bob)

LongTensor[_PointerTensor - id:9074118273 owner:local loc:bob id@loc:82371644971]

In [12]:
x_pointer_tensor_dict = {alice:x_alice, bob:x_bob}
y_pointer_tensor_dict = {alice:y_alice, bob:y_bob}

In [13]:
x_gp = gpt(x_pointer_tensor_dict,None, gpt)
y_gp = gpt(y_pointer_tensor_dict, None, gpt)

In [14]:
x_mpc = mpct(x_gp)
y_mpc = mpct(y_gp)

In [15]:
x_mpc

[_MPCTensor - id:2933021398 owner:local]

In [16]:
y_mpc

[_MPCTensor - id:2820382814 owner:local]

In [17]:
%%debug
z_mpc = x_mpc + y_mpc

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(2)<module>()

--Call--
> /home/bobby/Projects/OpenMined/PySyft/syft/core/frameworks/torch/mpc_tensor.py(37)__add__()
     35     # Put here all the methods you want to overload
     36 
---> 37     def __add__(self, other):
     38         # gp_ stands for GeneralizedPointer
     39         gp_response = spdz.spdz_add(self.shares, other.shares)

> /home/bobby/Projects/OpenMined/PySyft/syft/core/frameworks/torch/mpc_tensor.py(39)__add__()
     37     def __add__(self, other):
     38         # gp_ stands for GeneralizedPointer
---> 39         gp_response = spdz.spdz_add(self.shares, other.shares)
     40         response = _MPCTensor(gp_response)
     41         # response.shares = shares

--Call--
> /home/bobby/Projects/OpenMined/PySyft/syft/mpc/spdz.py(63)spdz_add()
     61 
     62 
---> 63 def spdz_add(a, b):
     64     return ((a + b) % field)
     65 

> /home/bobby/Projects/OpenMined/PySyft/syft/mpc/spdz.py(6